# Risk Aggregation POC - PLTs

This notebook reads can be used to read in multiple PLTs, rollup the PLTs to an aggregate PLT, and generate statistics and EP metrics from the PLTs.

-----

### Install necessary libraries

In [1]:
# %pip install pandas
# %pip install numpy
# %pip install pyarrow
# %pip install fastparquet


# Install to connect to AWS S3
# %pip install boto3
# %pip install botocore==1.22.5
# %pip install s3fs
# %pip install fsspec

### Import necessary libraries

In [2]:
import os
import sys
import pandas as pd
import numpy as np
import aggregationtools.plt
import aggregationtools
from aggregationtools.plt import PLT
from aggregationtools import plt_calculator
import glob

### Import data from AWS S3
AWS credentials may be provided explicitly with s3fs.S3FileSystem,
but it is more secure to exclude the credentials from the code.
Instead use the AWS CLI to `aws configure` credentials.

In [3]:
# import s3fs

# s3 = s3fs.S3FileSystem(anon=False)
# files = s3.glob('s3://fannie-mae-phase-3/03 Data Aggregation POC/01 Test/test_4/split/*.parquet')
# plts = pd.concat([pd.read_parquet('s3://' + fp) for fp in files])

### Import local data
Replace SCENARIO_FOLDER with scenario you'd like to run.

In [4]:
path = os.path.join('AggregationPOCScenarios', 'FLOOD_NEW_YORK', '')
files = glob.glob(path + '*.parquet')
plts = pd.concat([pd.read_parquet(fp) for fp in files])

### Create PLT from imported data
number_of_simulations to include number of period simulations and number of samples

In [5]:
json_test = plts.to_dict(orient='records')
original_plt = aggregationtools.plt.PLT(data = json_test, number_of_simulations = 800000*7)
original_plt.plt.head(5).style

,AccgrpId,LocName,Lob,LobId,CedantId,LossType,PeriodId,Weight,EventId,EventDate,LossDate,Loss
0,1520,0024cc59c00ac7bb6dc147ec9f564685e04,ATC1,1303,Unknown,All,892,0.000000,2856884,2020-08-15 00:00:00,2020-08-15 00:00:00,91.970487
1,1520,0024cc59c00ac7bb6dc147ec9f564685e04,ATC1,1303,Unknown,All,7652,0.000000,2856884,2020-08-24 00:00:00,2020-08-24 00:00:00,8.676204
2,1520,0024cc59c00ac7bb6dc147ec9f564685e04,ATC1,1303,Unknown,All,29267,0.000000,2856884,2020-08-15 00:00:00,2020-08-15 00:00:00,37.901831
3,1520,0024cc59c00ac7bb6dc147ec9f564685e04,ATC1,1303,Unknown,All,34795,0.000000,2872425,2020-09-30 00:00:00,2020-09-30 00:00:00,293.797882
4,1520,0024cc59c00ac7bb6dc147ec9f564685e04,ATC1,1303,Unknown,All,49586,0.000000,2862393,2020-09-28 00:00:00,2020-09-28 00:00:00,4.523303


See that number of sumulations has been set for the PLT:

In [6]:
original_plt.simulations

5600000

### Group account PLTs

In [7]:
grouped_plt = plt_calculator.group_plts(original_plt)
grouped_plt.plt.head(5).style

,PeriodId,EventId,EventDate,Weight,Loss
0,1,9350002,2020-04-02 00:00:00,0.000000,28.554314
1,5,9350117,2020-09-14 00:00:00,0.000000,86582.347290
2,8,2871193,2020-07-21 00:00:00,0.000000,330429.864529
3,11,10480776,2020-02-27 00:00:00,0.000000,27008.981184
4,16,2861444,2020-09-19 00:00:00,0.000000,678.495681


### Rollup PLT

In [8]:
rolled_up_plt = plt_calculator.roll_up_plts(original_plt)
rolled_up_plt.plt.head(5)

,PeriodId,Weight,Loss
0,1,1.785714e-07,28.554314
1,5,1.785714e-07,86582.347290
2,8,1.785714e-07,330429.864529
3,11,1.785714e-07,27008.981184
4,16,1.785714e-07,678.495681


### Calculate statistics from rolled up PLT

In [9]:
aal = rolled_up_plt.get_aal()
std = rolled_up_plt.get_standard_deviation()
covvar = rolled_up_plt.get_covvar()
print('AAL: ' + aal.astype(str))
print('STD: ' + str(std))
print('CovVar: ' + str(covvar))

AAL: 23697.326601278914
STD: 106996.69164219059
CovVar: 4.515137654237173


### Calculate AEP metrics from PLT

In [10]:
aep = plt_calculator.calculate_aep_curve(grouped_plt.plt, grouped_plt.simulations)
aep_metrics = aep.curve['Loss'][aep.curve['Loss'].index.isin([1/x for x in aep.RETURN_PERIODS])]
aep_metrics.index = np.reciprocal(aep_metrics.index)
aep_metrics = aep_metrics.rename_axis("Return Period").sort_index(axis=0, ascending=False)

# Return AEP results
aep_metrics.to_frame().style.format("{:,.0f}")

,Loss
Return Period,
10000.0,"2,088,304"
5000.0,"1,874,019"
1000.0,"1,286,494"
500.0,"1,035,294"
250.0,"803,379"
200.0,"729,597"
100.0,"541,243"
50.0,"365,427"
25.0,"160,455"


### Calculate OEP metrics from PLT

In [11]:
oep = plt_calculator.calculate_oep_curve(grouped_plt.plt, grouped_plt.simulations)
oep_metrics = oep.curve['Loss'][oep.curve['Loss'].index.isin([1/x for x in oep.RETURN_PERIODS])]
oep_metrics.index = np.reciprocal(oep_metrics.index)
oep_metrics = oep_metrics.rename_axis("Return Period").sort_index(axis=0, ascending=False)

# Return OEP results
oep_metrics.to_frame().style.format("{:,.0f}")

,Loss
Return Period,
10000.0,"1,971,769"
5000.0,"1,759,048"
1000.0,"1,232,628"
500.0,"998,653"
250.0,"767,405"
200.0,"696,533"
100.0,"523,739"
50.0,"355,579"
25.0,"152,056"
